**Configuración**

In [1]:
# Constantes globales.
ESPACIOS_TOTALES = 50
PRECIO_CUARTO = 500

# Datos iniciales. para el (Usuario admin)
Administrador = {"Admin": "UdeA1803"}

# Diccionarios para guardar usuarios, vehículos, parqueo...
Usuarios = {}
Vehiculos = {}
Estacion = {}
HistorialRetiros = []

# Usuario

In [2]:
# función para el registro de usuarios

def RegistrarUsuario():
    print("\n*Registrar*")
    Documento = input("Por favor ingrese el documento: ")

# Verificar si el usuario ya está registrado
    if Documento in Usuarios:
        print(f"El documento{Documento}, ya se encuentra registrado en el sistema.")
        return

# registrar datos
    Nombre = input("Ingrese el nombre: ").upper()
    Apellido = input("Ingrese el apellido: ").upper()
    Placa = input("Ingrese la matricula del vehículo: ").upper()

# Guardar la información en el diccionario 'Usuarios'
    Usuarios[Documento] = {
        "nombre": Nombre,
        "apellido": Apellido,
        "placa": Placa
    }
    print("El usuario ha sido registrado éxitosamente.")


# Registrar vehiculos

In [3]:
from datetime import datetime

# Constantes y estructuras
Estacion = {}  # clave: placa, valor: datos
EstacionesDisponibles = list(range(1, ESPACIOS_TOTALES + 1))

# Función para ingresar vehículo
def IngresarVehiculo():
    print("\n--- Ingreso de Vehículo ---")
    Documento = input("Documento de identidad del usuario: ")

# Validar si el usuario está registrado
    if Documento not in Usuarios:
        print("Usuario no registrado. Primero debe registrarlo.")
        return

# Localizar la placa asociada al documento
    Placa = Usuarios[Documento]["placa"]

# Verificar si el vehículo ya está estacionado
    if Placa in Estacion:
        print("Este vehículo ya está estacionado.")
        return

# Verificar si hay estaciones disponibles
# promesa, encontrar la forma de que esta estacione acorde al estudiante y la cercania de su facultad.
    if not EstacionesDisponibles:
        print("Lo sentimos, todas nuestras estaciones están llenas.")
        return

# Asignar la primera estación libre
    EstacionAsignada = EstacionesDisponibles.pop(0)

# Registrar la hora actual
    HoraIngreso = datetime.now()

# Almacenar información del ingreso
    Estacion[Placa] = {
        "documento": Documento,
        "nombre": Usuarios[Documento]["nombre"],
        "apellido": Usuarios[Documento]["apellido"],
        "estacion": EstacionAsignada,
        "hora_ingreso": HoraIngreso
    }

    print(f"Vehículo ingresado exitosamente. Estación asignada: {EstacionAsignada}\n")

# Retirar vehiculo

In [4]:
def generar_factura(placa, datos_usuario, hora_ingreso, hora_salida, minutos, costo):
    titulo = "FACTURA"
    ancho = 25
    print("\n" + "-" * ancho)
    print(titulo.center(ancho))
    print("-" * ancho)
    print(f"Placa:           {placa}")
    print(f"Celda:           {datos_usuario['estacion']}")
    print(f"Ingreso:         {hora_ingreso.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Salida:          {hora_salida.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Tiempo total:    {minutos} minutos")
    print(f"Total a pagar:   ${costo:,}")
    print("-" * ancho + "\n")

def RetirarVehiculo():
    print("\n--- Retiro de Vehículo ---")
    placa = input("Ingrese la placa del vehículo: ").upper()

    if placa not in Estacion:
        print("Vehículo no encontrado en el parqueadero.")
        return

    # Obtener datos del vehículo
    datos = Estacion[placa]
    hora_ingreso = datos["hora_ingreso"]
    hora_salida = datetime.now()
    duracion = hora_salida - hora_ingreso
    minutos = int(duracion.total_seconds() // 60)

    # Cálculo del cobro: mínimo se cobra una fracción (15 minutos = $300)
    cuartoH = max(1, (minutos + 14) // 15)  # Asegura al menos un bloque
    costo = cuartoH * PRECIO_CUARTO

    # Generar factura
    generar_factura(placa, datos, hora_ingreso, hora_salida, minutos, costo)

    # Liberar celda
    EstacionesDisponibles.append(datos["estacion"])
    EstacionesDisponibles.sort()  # Para asignar siempre en orden

    # Registrar retiro en historial
    HistorialRetiros.append({
        "placa": placa,
        "documento": datos["documento"],
        "nombre": datos["nombre"],
        "apellido": datos["apellido"],
        "celda": datos["estacion"],
        "ingreso": hora_ingreso.strftime('%Y-%m-%d %H:%M:%S'),
        "salida": hora_salida.strftime('%Y-%m-%d %H:%M:%S'),
        "tiempo_min": minutos,
        "costo": costo
    })

    # Eliminar de vehículos activos
    del Estacion[placa]


# Menú de admin

**ver usuarios registrados**

In [5]:
import pandas as pd

In [6]:
def VerUsuarios():
    print("\n--- Lista de Usuarios Registrados ---")
    if not Usuarios:
        print("No hay usuarios registrados.")
        return

    # Convertir el diccionario de usuarios en DataFrame
    df_usuarios = pd.DataFrame.from_dict(Usuarios, orient='index')
    df_usuarios.index.name = 'Documento'
    print(df_usuarios)

**ingresos**

In [7]:
def VerIngresos():
    print("\n Ingresos Totales \n")
    if not HistorialRetiros:
        print("No hay retiros registrados aún.")
        return

    total_ingresos = sum(retiro["costo"] for retiro in HistorialRetiros)
    print(f"El total de ingresos generados es: ${total_ingresos}\n")

**Disponibilidad** de celdas.

In [8]:
def VerOcupacion():
    ocupadas = len(Estacion)
    libres = len(EstacionesDisponibles)
    print("\n--- Disponibilidad del parqueadero ---")
    print(f"Celdas ocupadas: {ocupadas}")
    print(f"Celdas disponibles: {libres}\n")

**Resumen de vehículos estacionados**

In [9]:
def VerVehiculosParqueados():
    print("\n--- Vehículos en nuestra estación ---")
    if not Estacion:
        print("No hay vehículos en el estacionamiento.")
        return

    df = pd.DataFrame.from_dict(Estacion, orient='index')
    df = df[["nombre", "apellido", "documento", "estacion", "hora_ingreso"]]
    df["hora_ingreso"] = df["hora_ingreso"].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
    print(df)

**historial de retiros**

In [10]:
def ExportarHistorialRetiros():
    if not HistorialRetiros:
        print("No hay datos en el historial de retiros para exportar.")
        return

    df = pd.DataFrame(HistorialRetiros)
    nombre_archivo = "historial_retiros.csv"
    df.to_csv(nombre_archivo, index=False)
    print(f"Historial de retiros exportado correctamente a '{nombre_archivo}'.")

**Menú de admin**

In [11]:
#Menú de admin
def MenuAdministrador():
    print("\n--- Estación Admin ---")
    usuario = input("Ingrese usuario: ")
    contraseña = input("Ingrese la contraseña: ")

    if usuario in Administrador and Administrador[usuario] == contraseña:
        print("Acceso permitido.\n")
        while True:
            print("--- Opciones de Administrador ---")
            print("1. Disponibilidad del parqueadero")
            print("2. Ver usuarios registrados")
            print("3. Ver vehículos estacionados")
            print("4. Exportar historial de retiros")
            print("5. Ver Ingresos")
            print("6. Salir del administrador")
            opcion = input("Seleccione una opción: ")

            if opcion == "1":
                VerOcupacion()

            elif opcion == "2":
                  VerUsuarios()

            elif opcion == "3":
                VerVehiculosParqueados()

            elif opcion == "4":
                ExportarHistorialRetiros()

            elif opcion == "5":
                VerIngresos()

            elif opcion == "6":
                print("Saliendo del módulo de administración.\n")
                break
            else:
                print("Opción inválida. Intente de nuevo.\n")
    else:
        print("Usuario o contraseña incorrectos.\n")

# Menú General

In [13]:
def Menu():
    print("\n--- ESTACIONA UdeA ---")
    while True:
        print("1. Registrar")
        print("2. Ingresar vehículo")
        print("3. Retirar vehículo")
        print("4. Ingresar al módulo de administrador")
        print("5. Salir")

        opcion = input("Seleccione una opción: ")

        if opcion == "1":
            RegistrarUsuario()
        elif opcion == "2":
            IngresarVehiculo()
        elif opcion == "3":
            RetirarVehiculo()
        elif opcion == "4":
            MenuAdministrador()
        elif opcion == "5":
            print("Gracias por usar Parquea. Hasta luego.")
            break
        else:
            print("Opción no válida. Intente nuevamente.")

# Ejecutar programa
Menu()


--- ESTACIONA UdeA ---
1. Registrar
2. Ingresar vehículo
3. Retirar vehículo
4. Ingresar al módulo de administrador
5. Salir
Seleccione una opción: 1

*Registrar*
Por favor ingrese el documento: 123456789
Ingrese el nombre: sebastian
Ingrese el apellido: cortes
Ingrese la matricula del vehículo: mpi94g
El usuario ha sido registrado éxitosamente.
1. Registrar
2. Ingresar vehículo
3. Retirar vehículo
4. Ingresar al módulo de administrador
5. Salir
Seleccione una opción: 2

--- Ingreso de Vehículo ---
Documento de identidad del usuario: 123456789
Vehículo ingresado exitosamente. Estación asignada: 1

1. Registrar
2. Ingresar vehículo
3. Retirar vehículo
4. Ingresar al módulo de administrador
5. Salir
Seleccione una opción: 2

--- Ingreso de Vehículo ---
Documento de identidad del usuario: 1222454358
Usuario no registrado. Primero debe registrarlo.
1. Registrar
2. Ingresar vehículo
3. Retirar vehículo
4. Ingresar al módulo de administrador
5. Salir
Seleccione una opción: 1

*Registrar*
Po